## Modèle

Il faut d'abord lancer la cellule ci-dessous pour charger la variable "model" puis tester l'interface.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,cross_val_predict

from sklearn.metrics import mean_squared_error,r2_score,confusion_matrix,f1_score,precision_recall_curve,roc_curve,roc_auc_score
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml
import cv2

dataDict=fetch_openml('mnist_784', version=1)

print("Affichage des cles du dictionnaire : ")
for cle in dataDict.keys():
    print(cle)
    
print("Il y a {} features pour {} échantillons".format(dataDict["data"].shape[1],dataDict["data"].shape[0]))
X=dataDict['data']
print("Il y a {} targets".format(np.unique(dataDict['target']).size))
y=dataDict['target']
y=y.astype('int')
print("Les differentes valeurs sont : {}".format(np.unique(y)))
print("Description détaillée de la base de donnée :")
print(dataDict['DESCR'])

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=1/7,shuffle=False)
print("Il y a bien {} valeurs dans notre base d'apprentissage".format(x_train.shape[0]))

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np



targets_names = [0,1,2,3,4,5,6,7,8,9]


images_train = x_train.astype('float')/255
images_test = x_test.astype('float')/255

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))

model.add(tf.keras.layers.Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(256, 
                 kernel_size=(3, 3), 
                 activation='relu'))

model.add(tf.keras.layers.Conv2D(256, 
                 kernel_size=(3, 3), 
                 padding='same',
                 activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))

model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

images_train = images_train.reshape(-1,28,28,1)
images_test = images_test.reshape(-1,28,28,1)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history = model.fit(images_train,
                    y_train,
                    epochs=10,
                    validation_split=0.2)

## Interface graphique

L'interface nécessite ghostscript, il faut l'installer et régler les variables d'environnement pour éviter les problèmes.

Variable à créer :

`EpsImagePlugin.gs_windows_binary`

Contenu :

`C:\Program Files\gs\gs9.53.3\bin\;`

Il faut donc avoir téléchargé ghostscript ici (et l'avoir mis dans le bon répertoire pour le path) :
https://www.ghostscript.com/download/gsdnld.html


Sinon, installer ghostscript (au choix selon ce qui fonctionne) :

`conda install ghostscript`

`pip install ghostscript`

`conda install -c conda-forge ghostscript`

In [104]:
from tkinter import *
from tkinter import ttk, colorchooser
from PIL import Image as PIL_image
from matplotlib import pyplot as plt
import numpy as np
import cv2
import tensorflow as tf

'''
class Model:
    def __init__(self):
        self.model = tf.keras.models.load_model(".\saved_model\my_model")
'''

class App:
    def __init__(self):
        self.master = Tk()
        self.master.resizable(width=False, height=False)
        self.title = 'Chiffr-o-matic'
        self.color_fg = 'black'
        self.color_bg = 'white'
        self.old_x = None
        self.old_y = None
        self.canvas = None
        self.penwidth = 15
        self.prediction = StringVar()
        
        self.drawWidgets()
        # Binding du clic souris à la méthode paint :
        self.canvas.bind('<B1-Motion>',self.paint)
        # Binding du relâchement du bouton à la méthode reset pour arrêter de tracer :
        self.canvas.bind('<ButtonRelease-1>',self.reset)
        
    def paint(self,e):
        #Si les anciennes coordonnées existent, on commence à tracer (sinon elles sont crées après le if) :
        if self.old_x and self.old_y: 
            # Traçage de la ligne dans le canvas self.canvas depuis l'ancienne position à la nouvelle.
            self.canvas.create_line(self.old_x,
                               self.old_y,
                               e.x,
                               e.y,
                               width=self.penwidth,
                               fill=self.color_fg,
                               capstyle=ROUND,
                               smooth=True)
        # Les anciennes coordonnées sont rempies avec les points d'arrivée du coup de brush :
        self.old_x = e.x
        self.old_y = e.y

    def reset(self,e):    # RàZ de l'ancienne position x et y pour pouvoir "décoller" le brush entre deux strokes. 
        self.old_x = None
        self.old_y = None      

    def changeW(self,e): # Changement de l'épaisseur du brush avec le slider :
        self.penwidth = e
           

    def clear(self): # Nettoie tout le canvas.
        self.canvas.delete(ALL)

    def changer_fg(self):  # Changing the pen color
        self.color_fg=colorchooser.askcolor(color=self.color_fg)[1]

    def changer_bg(self):  # Changing the background color canvas
        self.color_bg=colorchooser.askcolor(color=self.color_bg)[1]
        self.canvas['bg'] = self.color_bg

    def predire(self):
        '''Affiche une prediction sur le chiffre en cours dans le canvas.'''
        self.enregistrer_image()
        # Chargement de l'image avec le paramètre 0 pour le niveau de gris uniquement :
        image = cv2.imread('image.png',0)

        # Redimensionnement de l'image au format 28 par 28 puis reshape au format accepté par le modèle :
        image = cv2.bitwise_not(image)
        image = cv2.resize(image, (28, 28))
        image = image.reshape(-1,28,28,1)

        y=model.predict(image.reshape(1,28,28,1))

        print(y[0])
        self.prediction = Label(self.controls, text=str(np.argmax(y[0])),font=('arial 18')).grid(row=1,column=0)
        self.controls.pack(side=LEFT)
        
    def enregistrer_image(self):
        '''Nécessite d'installer ghostscript via conda, permet d'enregistrer l'image du canvas en .eps
        puis de transformer ce .eps en .png avec PIL'''
        # Enregistrement au format postscipt : 
        self.canvas.postscript(file = "image.eps") 
        # Conversion en .png via PIL :
        img = PIL_image.open("image.eps") 
        img.save("image.png", "png")
        
    def drawWidgets(self):
        self.controls = Frame(self.master,padx = 5,pady = 5)
        Label(self.controls, text='Pen Width:',font=('arial 18')).grid(row=0,column=0)
        self.slider = ttk.Scale(self.controls,from_= 5, to = 100,command=self.changeW,orient=VERTICAL)
        self.slider.set(self.penwidth)
        self.slider.grid(row=0,column=1,ipadx=30)
        self.controls.pack(side=LEFT)
        
        self.canvas = Canvas(self.master,width=280,height=280,bg=self.color_bg,)
        self.canvas.pack(fill=BOTH,expand=True)

        menu = Menu(self.master)
        self.master.config(menu=menu)
        filemenu = Menu(menu)
        colormenu = Menu(menu)
        menu.add_cascade(label='Colors',menu=colormenu)
        colormenu.add_command(label='Brush Color',command=self.changer_fg)
        colormenu.add_command(label='Background Color',command=self.changer_bg)
        optionmenu = Menu(menu)
        menu.add_cascade(label='Options',menu=optionmenu)
        optionmenu.add_command(label='Predict',command=self.predire) 
        optionmenu.add_command(label='Clear Canvas',command=self.clear)
        optionmenu.add_command(label='Save',command=self.enregistrer_image) 
        optionmenu.add_command(label='Exit',command=self.master.destroy) 

#model = Model()
app = App()
app.master.mainloop()

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
